In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8,
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
X_train, X_test, y_train, y_test =train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

$TRAIN LOGISTIC REGRESSION$

In [4]:
log_reg = LogisticRegression(C=1, solver='liblinear')
log_reg.fit(X_train, y_train)
y_pred_log_reg = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log_reg))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       270
           1       0.85      0.57      0.68        30

    accuracy                           0.95       300
   macro avg       0.90      0.78      0.83       300
weighted avg       0.94      0.95      0.94       300



$RANDOM FOREST$

In [5]:
Rf_clf = RandomForestClassifier(n_estimators=30, max_depth=3)
Rf_clf.fit(X_train, y_train)
y_pred_rf = Rf_clf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98       270
           1       0.95      0.67      0.78        30

    accuracy                           0.96       300
   macro avg       0.96      0.83      0.88       300
weighted avg       0.96      0.96      0.96       300



$TRAIN  XGBoost$

In [8]:
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.96      0.73      0.83        30

    accuracy                           0.97       300
   macro avg       0.96      0.86      0.91       300
weighted avg       0.97      0.97      0.97       300



$HANDLE CLASS IMBALANCE USING SMOTETOMEK AND

In [10]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=0)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)

np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([616, 616], dtype=int64))

In [12]:

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print(classification_report(y_test, y_pred_xgb))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98       270
           1       0.96      0.73      0.83        30

    accuracy                           0.97       300
   macro avg       0.96      0.86      0.91       300
weighted avg       0.97      0.97      0.97       300



In [13]:
models = [
    (
        "Logistic Regression",
        LogisticRegression(C=1, solver='liblinear'),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest",
        RandomForestClassifier(n_estimators=30, max_depth=3),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [14]:
reports = []

for model_name, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
   
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [15]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [16]:
	
# Initialize MLflow
mlflow.set_experiment("imbalanced classification")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
   
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name)
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
       
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2025/03/08 22:32:56 INFO mlflow.tracking.fluent: Experiment with name 'imbalanced classification' does not exist. Creating a new experiment.
2025/03/08 22:32:56 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/849516178645292950/runs/ab461aceed9a4e078b26119b5e4bb9b8
🧪 View experiment at: http://localhost:5000/#/experiments/849516178645292950


2025/03/08 22:33:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/849516178645292950/runs/f4a4fd16bf9b45bbb6ad90d57c54887b
🧪 View experiment at: http://localhost:5000/#/experiments/849516178645292950


2025/03/08 22:33:27 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\SRIYAV~1\AppData\Local\Temp\tmpc8fstklu\model, flavor: xgboost). Fall back to return ['xgboost==2.1.3']. Set logging level to DEBUG to see the full traceback. 
2025/03/08 22:33:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/849516178645292950/runs/9e6195108d9a4a89894739a62b301803
🧪 View experiment at: http://localhost:5000/#/experiments/849516178645292950


2025/03/08 22:33:34 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\SRIYAV~1\AppData\Local\Temp\tmpl6ajftjy\model, flavor: xgboost). Fall back to return ['xgboost==2.1.3']. Set logging level to DEBUG to see the full traceback. 
2025/03/08 22:33:34 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/849516178645292950/runs/c8e8219ee405474e94d84ab2db94e402
🧪 View experiment at: http://localhost:5000/#/experiments/849516178645292950
